# AutoGen
### AutoGen is a framework that enables the development of LLM applications using multiple agents that can converse with each other to solve tasks. 
### See more details at https://github.com/microsoft/autogen

In [ ]:
## Installation
!pip install pyautogen

### AutoGen provides a drop-in replacement of openai.ChatCompletion as an enhanced inference API. 

In [ ]:
from autogen import oai
from openai.openai_object import OpenAIObject

ANYSCALE_ENDPOINT_API_KEY = "esecret_XXXXX"
response :OpenAIObject = oai.ChatCompletion.create(
    config_list=[
        {
            "model": "codellama/CodeLlama-34b-Instruct-hf",
            "api_type": "open_ai",
            "api_base": "https://console.endpoints.anyscale.com/m/v1",
            "api_key": ANYSCALE_ENDPOINT_API_KEY,
        }
    ],
    messages=[
        {"role": "system", "content": "you are an AI expert"},
        {"role": "user", "content": "Tell me a joke"}
    ],
    temperature=0.2
)

content = response.get("choices")[0].get("message").get("content")
print(content)

### Let create a multi-agent chat application with AutoGen
### 1, create a config list with your Anyscale Endpoint API Keys
#### Use **codellama/CodeLlama-34b-Instruct-hf** model for the best results

In [5]:
import autogen

config_list = [
    {
        "model": "codellama/CodeLlama-34b-Instruct-hf",
        "api_type": "open_ai",
        "api_base": "https://api.endpoints.anyscale.com/v1",
        "api_key": ANYSCALE_ENDPOINT_API_KEY
    }
]

### 2, create an LLM config with config list

In [6]:
llm_config = {
    "request_timeout": 600,
    "seed": 45,  # change the seed for different trials
    "config_list": config_list,
    "temperature": 0,
    "max_tokens": 2000,
}

### 3, Create an agent assistant

In [7]:
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config=llm_config,
    is_termination_msg=lambda x: True if "TERMINATE" in x.get("content") else False,
)

### 4, Create a user_proxy
#### To execute the code locally, you can use following **code_execution_config**,
#### create a dummy **.dockenv** file  by `touch /.dockerenv`,
#### and make sure **docker** is uninstalled by `pip uninstall docker`

In [15]:
# Set use_docker to False
code_execution_config = {"work_dir": "./local_test", "use_docker": None}

user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="TERMINATE",
    max_consecutive_auto_reply=1,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config=code_execution_config,
    llm_config=llm_config,
    system_message="""Reply TERMINATE if the task has been solved at full satisfaction.
Otherwise, reply CONTINUE, or the reason why the task is not solved yet.""")

### 5, Start your chat with AutoGen
#### You can have human input at the end of each round of agent conversation
#### and enter **exit** to finish the chat.

In [17]:
task1 = "What time is it right now? and how many hours till midnight?"
user_proxy.initiate_chat(assistant,message=task1)

user_proxy (to assistant):

What time is it right now? and how many hours till midnight?

--------------------------------------------------------------------------------
assistant (to user_proxy):

  To get the current time and calculate the hours till midnight, you can use the following code:
```python
import datetime

# Get the current time
now = datetime.datetime.now()

# Calculate the hours till midnight
hours_till_midnight = (24 - now.hour) % 24

# Print the result
print(f"It is currently {now.strftime('%I:%M %p')}.")
print(f"There are {hours_till_midnight} hours till midnight.")
```
This code uses the `datetime` module to get the current time and calculate the hours till midnight. The `strftime` method is used to format the time in a human-readable format.

You can execute this code in a Python interpreter or save it to a file and run it with the Python interpreter.

Note: The `datetime` module is a part of the Python standard library, so you don't need to install any additional

execute_code was called without specifying a value for use_docker. Since the python docker package is not available, code will be run natively. Note: this fallback behavior is subject to change


user_proxy (to assistant):

exitcode: 0 (execution succeeded)
Code output: 
It is currently 06:59 PM.
There are 6 hours till midnight.


--------------------------------------------------------------------------------
assistant (to user_proxy):

  Thank you for letting me know. I apologize for the confusion earlier. It's great to hear that the code executed successfully and provided the correct output. If you have any more questions or need further assistance, feel free to ask!

--------------------------------------------------------------------------------


Please give feedback to assistant. Press enter to skip and use auto-reply, or type 'exit' to stop the conversation:  exit
